## PROYECTO SPRINT 10
---

### 1. Descripción del proyecto 
Se ha decidido abrir un pequeño café regentado por robots en Los Ángeles. Si bien el proyecto presenta un potencial prometedor, también conlleva un costo significativo. Por ello, se ha optado por explorar estrategias para atraer inversionistas que compartan el interés en las dinámicas actuales del mercado.

Para fortalecer la propuesta de inversión, se llevará a cabo un estudio de mercado exhaustivo utilizando datos recopilados de fuentes abiertas sobre el panorama de restaurantes en Los Ángeles. Este análisis permitirá comprender a fondo las tendencias del sector, identificar oportunidades de mercado y delinear una estrategia sólida para el éxito del café.

---
### 2. Carga de dataset y procesamiento de datos 

In [5]:
# Importación de librerías neceserarias
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import re

In [8]:
#Asignación de variable al dataset

def load_data():
    try:
        # Intentar cargar los datos de la forma local
        rest_data = pd.read_csv('rest_data_us_upd.csv') # Reemplaza con la ruta local real
        return rest_data
    except FileNotFoundError:
        # Si la carga local falla, intentar la carga desde la instancia de revisión
        try:
            rest_data = pd.read_csv('/datasets/rest_data_us_upd.csv')
            return rest_data
        except FileNotFoundError:
            print("No se pudo encontrar el archivo en ninguna de las ubicaciones.")
            return None

# Llamar a la función para cargar los datos
rest_data = load_data()

In [9]:
#Visualización de la información del dataset
rest_data.info()
rest_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 452.5+ KB


,id,object_name,address,chain,object_type,number
7085,18871,PHO 2000 RESTAURANT,2897 W OLYMPIC BLVD STE #103,True,Restaurant,47
4922,16708,CLARITA'S RESTAURANT,5024 E HUNTINGTON DR S,True,Restaurant,24
7522,19308,STAR STRIP,365 N LA CIENEGA BLVD,False,Restaurant,136
6642,18428,BODHI BOWL,645 W 9TH ST STE #107,False,Restaurant,13
3887,15673,CAKE HOUSE,440 S VERMONT AVE STE #112,True,Restaurant,20
4263,16049,BAR AMA,118 W 4TH ST,False,Bar,141
1532,13318,MARISCOS SABOR LATINOS,2201 S SAN PEDRO ST,False,Restaurant,32
8953,20739,SNOCIETY URBAN EATERY,330 E 2ND ST UNIT C,False,Restaurant,34
5716,17502,CBC EAGLE ROCK,1351 COLORADO BLVD,True,Restaurant,12
479,12265,SILVERLAKE WINE COMPANY,2395 N GLENDALE BLVD STE A,False,Restaurant,26


Al analizar el dataset, tanto a través de la visualización de su información como de una muestra representativa, podemos identificar el tipo de datos de cada columna y detectar la presencia de valores ausentes.

En el caso particular de la columna 'chain', observamos que, presenta una baja cantidad de valores ausentes y el tipo de dato asignado no es el adecuado.

Por lo tanto, se procederá a realizar las modificaciones necesarias para garantizar la correcta manipulación y análisis de la información contenida en esta columna 

In [ ]:
#Se identifican los valores ausentes de la columna 'chain'
missing_values = rest_data['chain'].isnull()
rows_with_missing_values = rest_data[missing_values]
rows_with_missing_values

,id,object_name,address,chain,object_type,number
7408,19194,TAQUERIA LOS 3 CARNALES,5000 E WHITTIER BLVD,NaN,Restaurant,14
7523,19309,JAMMIN JIMMY'S PIZZA,1641 FIRESTONE BLVD,NaN,Pizza,1
8648,20434,THE LEXINGTON THEATER,129 E 3RD ST,NaN,Restaurant,35


In [ ]:
# Cambio de tipo de dato en la columna 'chain'
rest_data['chain'] = rest_data['chain'].astype(bool)

rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9651 non-null   bool  
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 386.5+ KB


Después de realizar el cambio del tipo de dato de de la columna 'chain', se puede observar que ya no existen valores ausentes en la misma columna. 

---
### 3. Análisis de datos 

In [ ]:
# Búsqueda de las proporciones de los distintos tipos de establecimientos.
establishments= rest_data.groupby('object_type').agg({'object_name':'count'})
establishments.sort_values(by='object_name', ascending=False)

,object_name
object_type,
Restaurant,7255
Fast Food,1066
Cafe,435
Pizza,320
Bar,292
Bakery,283


In [ ]:
#Gráfico para visualizar proporción de los distintos tipos de establecimientos.
labels = establishments.index.to_list()
object_name_values = establishments['object_name'].values
df = px.data.tips()
fig = px.pie(df, values=object_name_values, names=labels, color_discrete_sequence=px.colors.sequential.RdBu, 
             title='Proporciones de los diferentes tipos de establecimientos')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Como podemos apreciar en el gráfico,  el establecimiento tipo 'Restaurant' cuenta con el mayor porcentaje con el 75.2%, seguido de 'Fast Food' con el 11%. Teniendo entre una diferencia bastante significativa entre ambos y muy alejados el uno del otro. En último lugar, y no tan alejado, lo ocupa 'Bakery' con un 2.93%.

In [ ]:
#Búsqueda de las proporciones de los establecimientos que pertenecen a una cadena y de los que no.
establishments= rest_data.groupby('chain').agg({'object_name':'count'})
establishments

,object_name
chain,
False,5972
True,3679


In [ ]:
#Gráfico para visualizar proporción de establecimientos que pertenecen a una cadena.
labels = ['Franquicia','Independiente']
object_name_values = [3679, 5972]
df = px.data.tips()
fig = px.pie(df, values=object_name_values, names=labels, color_discrete_sequence=px.colors.sequential.RdBu, 
             title='Proporción total de establecimientos que pertenecen a una cadena')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Basándose en nuestro gráfico el 61.9% son establecimientos independientes, es decir, no pertenecen a ninguna franquicia. El 38.1% pertenecen a una cadena.

In [ ]:
# Búsqueda de establecimiento que son habitualmente una cadena.
establishments_chain= rest_data.groupby('object_type').agg({'chain':'sum'})
establishments_chain= establishments_chain.sort_values(by='chain', ascending=False)
establishments_chain

,chain
object_type,
Restaurant,2294
Fast Food,605
Bakery,283
Cafe,266
Pizza,154
Bar,77


In [ ]:
# Gráfico para visualizar proporción de establecimientos que pertenecen a una cadena.
labels = establishments_chain.index.to_list()
chain_values = establishments_chain['chain'].values

df = px.data.gapminder()
fig = px.bar(df, y=chain_values, x=labels, text_auto='.2s',
            title="Número de cadenas por tipo de establecimmiento", 
            color_discrete_sequence=px.colors.sequential.RdBu,
            template='simple_white')

plt.tight_layout()
fig.show()

<Figure size 640x480 with 0 Axes>

Los establecimientos tipo 'Restaurant' pertenecen mayormente a franquicias con 2,294 lugares, por otro lado, el establecimiento tipo 'Bar' es el que menor cantidad de franquicias tiene, con solo 77. 

In [ ]:
# ¿Qué caracteriza a las cadenas: muchos establecimientos con un pequeño número de asientos o unos pocos establecimientos con un montón de asientos?

# Encontrar promedio de asientos en establecimientos que son cadena
establishments_chain_seats= rest_data[rest_data['chain'] == True]
print('Número promedio de asientos:', establishments_chain_seats['number'].mean())

# Encontrar número de asientos mayor al promedio 
establishments_chain_seats_= establishments_chain_seats[establishments_chain_seats['number'] > 39]
print('Número de establecimientos con más de 39 asientos:', establishments_chain_seats_['number'].count())

# Encontrar número de asientos menor al promedio 
establishments_chain_seats= establishments_chain_seats[establishments_chain_seats['number'] < 39]
print('Número de establecimientos con menos de 39 asientos:', establishments_chain_seats['number'].count())

Número promedio de asientos: 39.675455286762705
Número de establecimientos con más de 39 asientos: 1131
Número de establecimientos con menos de 39 asientos: 2505


La respuesta a la pregunta: ¿Qué caracteriza a las cadenas: muchos establecimientos con un pequeño número de asientos o unos pocos establecimientos con un montón de asientos? Es, muchos establecimientos con pocos asientos. Se llego a esta conclusión buscando primero el promedio de número de asientos que tienen los establecimientos que son cadena, después se buscó el número de los establecimientos que tenían más y menos de la cantidad promedio. El resultado fue que existen más establecimientos con menos cantidad de asientos. 




In [ ]:
#Búsqueda del promedio de número de asientos para cada tipo de restaurante. 
establishments= rest_data.groupby('object_type').agg({'number':'mean'})
establishments=establishments.sort_values(by='number', ascending=False)
establishments

,number
object_type,
Restaurant,48.042316
Bar,44.767123
Fast Food,31.837711
Pizza,28.459375
Cafe,25.000000
Bakery,21.773852


In [ ]:
# Gráfico para visualizar el promedio de número de asientos para cada tipo de restaurante. 
labels = establishments.index.to_list()
chain_values = establishments['number'].values

df = px.data.gapminder()
fig = px.bar(df, y=chain_values, x=labels, text_auto='.2s',
            title="Promedio de número de asientos por tipo de establecimiento", 
            color_discrete_sequence=px.colors.sequential.RdBu,
            template='simple_white')

plt.tight_layout()
fig.show()

<Figure size 640x480 with 0 Axes>

Los establecimientos tipo 'Restaurant' tienen el promedio más alto de asientos con 48, en segundo lugar, con poca diferencia continua 'Bar' con 45. Y en el último lugar 'Bakery' con 22 asientos promedio.

In [ ]:
#Coloca los datos de los nombres de las calles de la columna address en una columna separada.
rest_data['street_name']= pd.NA

for index, row in rest_data.iterrows():
    address = row['address']

    street_name_match = re.search(r"^(?:\d+)(?:\s+)?(?P<street_name>.+)$", address)

    if street_name_match:
        street_name = street_name_match.group('street_name')
        rest_data.loc[index, 'street_name'] = street_name
    else:
        street_name = "Not Found"

rest_data.head()

,id,object_name,address,chain,object_type,number,street_name
0,11786,HABITAT COFFEE SHOP,3708 N EAGLE ROCK BLVD,False,Cafe,26,N EAGLE ROCK BLVD
1,11787,REILLY'S,100 WORLD WAY 120,False,Restaurant,9,WORLD WAY 120
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD 253,False,Fast Food,20,HOLLYWOOD BLVD 253
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22,W SUNSET BLVD
4,11790,POLLEN,2100 ECHO PARK AVE,False,Restaurant,20,ECHO PARK AVE


In [ ]:
#Búsqueda de las 10 mejores calles por números de restaurantes.
top_10_streets = rest_data[rest_data['object_type']== 'Restaurant']
top_10_streets= top_10_streets.groupby('street_name').agg({'object_type':'count'})
top_10_streets= top_10_streets.sort_values(by='object_type', ascending=False).head(10)
top_10_streets

,object_type
street_name,
W PICO BLVD,218
W SUNSET BLVD,217
HOLLYWOOD BLVD,136
WILSHIRE BLVD,131
W 3RD ST,119
W OLYMPIC BLVD,107
SANTA MONICA BLVD,107
S VERMONT AVE,107
MELROSE AVE,106


In [ ]:
# Gráfico para visualizar las 10 mejores calles por números de restaurantes. 
labels = top_10_streets.index.to_list()
values = top_10_streets['object_type'].values

df = px.data.gapminder()
fig = px.bar(df, y=values, x=labels, text_auto='.2s',
            title="Top 10: Calles con más restaurentes", 
            color_discrete_sequence=px.colors.sequential.RdBu,
            template='simple_white')

plt.tight_layout()
fig.show()

<Figure size 640x480 with 0 Axes>

En el gráfico anterior se puede observar, que las calles W PICO BLVD y W SUNSET BLVD tienen la cantidad más alta de restaurantes con 218 y 217, son las únicas calles con más de 200. En decimo lugar se encuentra BEVERLY BLVD con 100 restaurantes. 

In [ ]:
#Encontrar el número de calles que solo cuentan con un restaurant 
one_r_street = rest_data[rest_data['object_type']== 'Restaurant']
one_r_street = one_r_street.groupby('street_name').agg({'object_type':'count'})
one_r_street= one_r_street.sort_values(by='object_type', ascending=True)
one_r_street= one_r_street[one_r_street['object_type'] == 1]

print('Número de calles que solo cuentan con un restaurant:', one_r_street['object_type'].count())

Número de calles que solo cuentan con un restaurant: 1969


In [ ]:
#Distribución del número de asientos
top_streets = rest_data[rest_data['object_type']== 'Restaurant']
top_streets= top_streets.groupby('street_name').agg({'object_type':'count', 'number':'sum'})
top_streets= top_streets.sort_values(by='number', ascending=False).head(10)
top_streets.reset_index()

,street_name,object_type,number
0,W SUNSET BLVD,217,12088
1,W PICO BLVD,218,9857
2,WILSHIRE BLVD,131,8786
3,HOLLYWOOD BLVD,136,7948
4,W OLYMPIC BLVD,107,6382
5,MELROSE AVE,106,5853
6,W 3RD ST,119,5671
7,BEVERLY BLVD,103,5343
8,S FIGUEROA ST,94,5316
9,S VERMONT AVE,107,5119


In [ ]:
labels = top_streets.index.to_list()
values = top_streets['number'].values

df = px.data.gapminder()
fig = px.bar(df, y=values, x=labels, text_auto='.2s',
            title="Distribución de asientos en calles del top 10", 
            color_discrete_sequence=px.colors.sequential.RdBu,
            template='simple_white')

plt.tight_layout()
fig.show()

<Figure size 640x480 with 0 Axes>

En el gráfico anterior se puede notar que la calle W SUNSET BLVD es la que tiene más asientos del top 10 con calles, en esta ocasión la calle W PICO BLVD tiene el segundo lugar con una diferencia significativa. En último lugar quedó la calle S VERMONT AVE con solo 5119, un poco menos de la mitad de la calle posicionada en primer lugar. 

---
### 4. Presentación

Presentation: <https://drive.google.com/file/d/1ONAH9NPRFPetZz_2rZp3B-zAeA0RPhlk/view?usp=sharing>